In [ ]:
import numpy as np
from view_sampler import ViewSampler, CameraConfig
from manipulated_object import ObjectPosition
from utils.orient import OrientUtils

cam_config = CameraConfig(location=(0, 0, 0.1), rotation=(np.pi / 2, 0, 0), fov=30)
world_viewer = ViewSampler("data/world_mug.xml", cam_config, simulation_time=0)
sim_viewer = ViewSampler("data/world_mug_sim.xml", cam_config)

init_location = (0, 1.3, 0.3)
random_orientations = OrientUtils.generate_random(200)
eval_positions = [ObjectPosition(orient, init_location) for orient in random_orientations]

In [ ]:
from algs.uniform_sampling import UniformSampling
from evaluate.evaluator import Evaluator
from evaluate import eval_funcs
import loss_funcs
import cv2 as cv


loss_functions = [
    loss_funcs.IOU(),
    loss_funcs.MSE(),
    loss_funcs.NormMSE(norm="euclidean"),
    loss_funcs.MutualInformation(bins=100),
    loss_funcs.PeakSignalNoiseRation(),
    loss_funcs.StructuralSimilarity(win_size=None),
    loss_funcs.HausdorffDistance(),
    loss_funcs.AdaptedRandError(),
    loss_funcs.VariationOfInformation(),
]

alg_config = UniformSampling.Config(time_limit=1000, min_samples=343, randomized=False, silent=True)

evaluator = Evaluator(world_viewer, eval_func=eval_funcs.NormXorDiff(obj_depth=2.0, method="mse"))

results = dict()

for loss in loss_functions:
    alg = UniformSampling(sim_viewer, loss_func=loss)
    eval_losses = evaluator.evaluate(alg, alg_config, eval_positions)
    results[type(loss).__name__] = eval_losses
    print(f"{type(loss).__name__}: {eval_losses}")

cv.destroyAllWindows()

In [ ]:
print(results)